## Install required packages for data cleaning

In [ ]:
!pip install -U textblob
!python -m textblob.download_corpora
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install emoji --upgrade

## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

## Load datasets

In [ ]:
location1 = pd.read_excel(r"location.xlsx")
location2 = pd.read_excel(r"location1.xlsx")
df = pd.read_excel(r"tweets.xlsx")

In [ ]:
location = pd.concat([location1,location2])

In [ ]:
location.info()

In [ ]:
merge = pd.merge(df, location, how = 'left', on = 'screen_name')

In [ ]:
merge.info()

In [ ]:
# % of missing.
for col in merge.columns:
    pct_missing = np.mean(merge[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

## Data Cleaning

In [ ]:
# remove duplicates
merge.drop_duplicates(subset=["text","timestamp"], keep='first', inplace=True)

In [ ]:
merge.dropna(subset=['location'], inplace=True)

In [ ]:
merge = merge.drop(merge[merge.retweets > 0].index)

In [ ]:
merge=merge.drop(['Unnamed: 0_x','Unnamed: 0_y','has_media', 'img_urls', 'is_replied',
       'is_reply_to', 'likes', 'links', 'parent_tweet_id', 'replies',
       'reply_to_users','text_html','tweet_id', 'tweet_url', 'user_id','video_url'], axis=1)

In [ ]:
merge.info()

In [ ]:
import re 
from textblob import TextBlob 
import emoji  

def clean_tweet(text): 
    text = re.sub(r'@[A-Za-z0-9]+', '', str(text)) # remove @mentions
    text = re.sub(r'#', '',  str(text)) # remove the '#' symbol
    text = re.sub(r'RT[\s]+', '',  str(text)) # remove RT
    text = re.sub(r'https?\/\/S+', '',  str(text)) # remove the hyperlink
    text = re.sub(r'http\S+', '',  str(text)) # remove the hyperlink
    text = re.sub(r'www\S+', '',  str(text)) # remove the www
    text = re.sub(r'twitter+', '',  str(text)) # remove the twiiter
    text = re.sub(r'pic+', '',  str(text)) # remove the pic
    text = re.sub(r'com', '',  str(text)) # remove the pic

    return text

def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)



In [ ]:
merge['text'] = merge['text'].apply(clean_tweet)

In [ ]:
merge['text'] = merge['text'].apply(remove_emoji)

In [ ]:
merge['text'] = [text.lower() for text in merge['text']]


In [ ]:
def getSubjectivity(text):
    return TextBlob( str(text)).sentiment.subjectivity

def getPolarity(text):
    return TextBlob( str(text)).sentiment.polarity

# merge['Subjectivity'] = merge['text'].apply(getSubjectivity)
# merge['Polarity'] = merge['text'].apply(getPolarity)

In [ ]:
merge['text']

In [ ]:
# load library
from nltk.corpus import stopwords

# download the set of stop words the first time
import nltk
nltk.download('stopwords')

In [ ]:
# Load stop words
stop_words = stopwords.words('english')

# Show stop words
stop_words[:10]

In [ ]:
# Remove stop words
merge['clean_tweets'] = merge['text'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop_words))

In [ ]:
merge['clean_tweets']

In [ ]:
merge

In [ ]:
# convert to lower case
merge['location'] = merge['location'].str.lower()
# remove trailing white spaces
merge['location'] = merge['location'].str.strip()

In [ ]:
def clean_location(text): 
    # text = re.sub(r'[@?|$|.|!|0-9|°|\|/|#|()]', '', str(text)) # remove not a letters 
    text = re.sub(r'[^a-zA-Z]+', ' ', str(text)) # remove not a letters 
    return text


In [ ]:
merge['clean_location'] = merge['location'].apply(clean_location)


In [ ]:
merge['clean_location'] = merge['clean_location'].apply(remove_emoji)

In [ ]:
merge.to_excel("all.xlsx") 